In [5]:
import struct
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import csv
import os
#import porn
from os import listdir
from os.path import isfile, join
from pathlib import Path
import math

In [6]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
def find2(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return root, name

# create readable structure from txt file
def create_frame_info(fi_name):
    with open(fi_name) as f:
        frame_info = f.readlines()
    return frame_info

# create a list of ids present in a readable structure
def get_ids_from_txt(frame_info):
    ids = []
    for line in frame_info:
        a, rest = line.split(",", 1)
        ids.append(int(a))
    return(ids)

#get set difference
def get_difference(a, b):
    return list(set(a).difference(set(b)))

def remove_rows(dframe, idlist):
    dframe = dframe.set_index("detection_id")
    dframe = dframe.drop(idlist, axis=0)
    return dframe

# get all points from a contour and store as a list of tuples
def get_contour_points(contour):
    cont = contour[2:-1]
    tuples = cont.split(";")
    l = []
    for item in tuples:
        a, b = item.split(" ")
        l.append((int(a), int(b)))
    return l

def process_image(img, x, y, cont_points):
    mask = np.zeros((100,100), dtype=int)
    xdiff = x - 10
    ydiff = y - 10
    xs = []
    ys = []
    for (a,b) in cont_points:
        v = a - xdiff
        u = b - ydiff
        xs.append(u)
        ys.append(v)
        img[u,v] = (0,0,255)
        mask[u,v] = 1
    return img, mask, xs, ys

def normalizedRGB(img):
    newimg=np.zeros((img.shape))
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            b,g,r = col
            s = b+g+r
            newimg[i,j] = (b//s), (g//s), (r//s)
    return newimg        

def com(img, startpoint, mask):
    (xavg, yavg) = startpoint
    
    nextleft = (xavg, yavg-1)
    nextright = (xavg, yavg+1)
    nextup = (xavg+1, yavg)
    nextdown = (xavg-1, yavg)
    
    if (mask[nextleft]==0):
        mask[nextleft] = 1
        com(img, nextleft, mask)
    if (mask[nextright]==0):
        mask[nextright] = 1
        com(img, nextright, mask)
    if (mask[nextup]==0):
        mask[nextup] = 1
        com(img, nextup, mask)
    if (mask[nextdown]==0):
        mask[nextdown] = 1
        com(img, nextdown, mask)

In [7]:
with open('csvnames.txt', 'r') as f:
    csvfiles = f.readlines()

In [8]:
# static paths to csvs, videos, frame infos, and npys
path_to_csvs = "/media/sam/Sam\'s Drive/SUBSET/SQL/ALLYEARS/"
videos_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
frame_info_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
npy_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/final/"

In [11]:
print("Filename\t\t\t\t\tStatus\t\t% Done\tFrames saved".format())
total = 4521
done = 0
index_error = []
others = []
percent = 0
for csvname in csvfiles:
    try:
        saved_imgs = 0
        # format name to get full path to csv
        csv_path = path_to_csvs + csvname[:-1]

        # create initial dataframe
        df = pd.read_csv(csv_path)
        # get video ID from current df
        videoID = df.iloc[0][4][1:]
        apath = "output/a/"+videoID
        bpath = "output/b/"+videoID
        os.mkdir(apath)
        os.mkdir(bpath)
        # get names for files
        video_name_fix = "summary_" + videoID + ".avi"
        npy_name_fix = videoID + ".RESULT.npy"
        frame_info_fix = "frame_info_" + videoID + ".txt"

        # get paths to video, frame info and npy
        videopath, videoname = find2(video_name_fix, videos_dir)
        video_path = videopath + "/" + videoname
        frame_info_path = find(frame_info_fix, frame_info_dir)
        npy_path = find(npy_name_fix, npy_dir)

        # get ids from frame info file and from df, remove rows from df that dont exist in frame info
        frame_info_file = create_frame_info(frame_info_path)
        frame_info_ids = get_ids_from_txt(frame_info_file)
        df_ids = df["detection_id"].tolist()
        ids_to_remove = get_difference(df_ids, frame_info_ids)
        clean_df = remove_rows(df, ids_to_remove)

        # load npy file, this is used together with video to produce images
        npy = np.load(npy_path)
        vidcap = cv2.VideoCapture(video_path)
        success,image = vidcap.read()
        # counts every iteration
        count = 0
        # initialize df where we will save stats
        stats = pd.DataFrame(columns=["B", "G", "R", "nB", "nG", "nR", "Compactness"])
        stats.index.name = "detection_id"

        while success:
            if (npy[count] and df.iloc[count][' specie_id'] == 0):
        #         print("Normal image:")
        #         plt.imshow(image)
        #         plt.show()

                index = str(clean_df.index[count])
                # save original image
                imgpath = "output/a/"+videoID+"/"+index+".jpg"
                cv2.imwrite(imgpath, image)
                contour = clean_df.iloc[count][13]
                x = clean_df.iloc[count][5]
                y = clean_df.iloc[count][6]
                contour_points = get_contour_points(contour)
                cont_img, mask, xs, ys = process_image(image, x, y, contour_points)
                # save image with contour
                contpath = "output/b/"+videoID+"/"+index+".jpg"
        #         print("Contoured image:")
        #         plt.imshow(cont_img)
        #         plt.show()


                a_fish = np.bincount(mask.reshape((10000,)))[1]
                p_fish = len(contour_points)
                r = p_fish / (2*math.pi)
                a_circle = math.pi * r * r
                comp = a_fish / a_circle

                xavg = int(np.mean(xs))
                yavg = int(np.mean(ys))
                # set coordinates for CoM point
                startpoint = (xavg, yavg)

                # compute CoM-based shape
                try:
                    com(cont_img, startpoint, mask)            
                    cv2.imwrite(contpath, cont_img)
                except IndexError:
                    print("{}\tIndexE\t\t{:2.2f}% left\t{}".format(csvname[:-1], percent, saved_imgs))
                    count+= 1
                    success, image = vidcap.read()
                    continue
                # extract specific channels
                channel0 = image[:,:,0]
                channel1 = image[:,:,1]
                channel2 = image[:,:,2]
                # compute averages
                c0avg = np.mean(mask*channel0)
                c1avg = np.mean(mask*channel1)
                c2avg = np.mean(mask*channel2)
                # compute normalized image
                normRGB = normalizedRGB(image)
                # extract normalized RGB channels
                normc0 = normRGB[:,:,0]
                normc1 = normRGB[:,:,1]
                normc2 = normRGB[:,:,2]
                # compute averages
                normc0avg = np.mean(mask*normc0)
                normc1avg = np.mean(mask*normc1)
                normc2avg = np.mean(mask*normc2)
                # save to dataframe
                stats.loc[index] = [c0avg, c1avg,c2avg, normc0avg, normc1avg, normc2avg, comp]
                saved_imgs+=1
            count+= 1
            success, image = vidcap.read()

        csvpath = "output/csv/"+videoID+".csv"
        stats.to_csv(csvpath)
        done+=1
        percent = done / total
        print("{}\tDone\t\t{:2.2f}% left\t{}".format(csvname[:-1], percent, saved_imgs))
        
    except Exception as e:
        print("{}\t{}".format(csvname[:-1], e))
        others.append(csvname[:-1])
        continue

Filename					Status		% Done	Frames saved


/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in ubyte_scalars
/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in ubyte_scalars


data_siteHoBiHu_camera1_201008230800.csv	Done		0.00% left	3
data_siteHoBiHu_camera1_201008240800.csv	Done		0.00% left	5
data_siteHoBiHu_camera1_201008260800.csv	IndexE		0.00% left	12
data_siteHoBiHu_camera1_201008260800.csv	Done		0.00% left	24
data_siteHoBiHu_camera1_201008270800.csv	Done		0.00% left	19
data_siteHoBiHu_camera1_201008280800.csv	Done		0.00% left	16
data_siteHoBiHu_camera1_201008290800.csv	Done		0.00% left	60
data_siteHoBiHu_camera1_201009020800.csv	Done		0.00% left	11
data_siteHoBiHu_camera1_201009030800.csv	Done		0.00% left	38
data_siteHoBiHu_camera1_201009040800.csv	Done		0.00% left	34
data_siteHoBiHu_camera1_201009060800.csv	Done		0.00% left	1
data_siteHoBiHu_camera1_201009070800.csv	Done		0.00% left	28
data_siteNPP-3_camera1_201203290800.csv	cannot unpack non-iterable NoneType object
data_siteNPP-3_camera1_201203310800.csv	cannot unpack non-iterable NoneType object
data_siteNPP-3_camera1_201204020800.csv	cannot unpack non-iterable NoneType object
data_siteNPP-3_camer

data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	362
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	401
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	411
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	426
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	436
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	440
data_siteNPP-3_camera3_201210070800.csv	IndexE		0.01% left	495
data_siteNPP-3_camera3_201210070800.csv	maximum recursion depth exceeded in comparison
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	1
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	21
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	31
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	77
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	92
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	95
data_siteNPP-3_camera3_201210080800.csv	IndexE		0.01% left	100
data_siteNPP-3_camera3_201210080800.cs

data_siteNPP-3_camera3_201210150800.csv	Done		0.01% left	530
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	2
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	55
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	86
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	141
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	145
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	239
data_siteNPP-3_camera3_201210160800.csv	IndexE		0.01% left	298
data_siteNPP-3_camera3_201210160800.csv	Done		0.01% left	305
data_siteNPP-3_camera3_201210170800.csv	IndexE		0.01% left	20
data_siteNPP-3_camera3_201210170800.csv	IndexE		0.01% left	121
data_siteNPP-3_camera3_201210170800.csv	IndexE		0.01% left	148
data_siteNPP-3_camera3_201210170800.csv	Done		0.01% left	170
data_siteNPP-3_camera3_201210180800.csv	IndexE		0.01% left	6
data_siteNPP-3_camera3_201210180800.csv	IndexE		0.01% left	6
data_siteNPP-3_camera3_201210180800.csv	IndexE		0.01% left	15
data_sit

...

data_siteNPP-3_camera1_201104230800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201104240800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201104260800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201104270800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201104280800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201104300800.csv	cannot unpack non-iterable NoneType object
data_siteNPP-3_camera1_201105020800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201105030800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201105040800.csv	Done		0.54% left	0
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	0
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	0
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	0
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	13
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	13
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	13
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	18
data_siteNPP-3

data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1371
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1371
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1371
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1448
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1448
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1448
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1448
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1451
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1525
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1607
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1607
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1607
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1669
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1687
data_siteNPP-3_camera1_201105060800.csv	IndexE		0.54% left	1687
data_siteNPP-3_camera1_201105060800.csv	

data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	211
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	216
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	220
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	222
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	227
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	229
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	237
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	254
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	266
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	283
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	299
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	313
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	345
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	345
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% left	345
data_siteNPP-3_camera4_201102030800.csv	IndexE		0.55% l

data_siteNPP-3_camera3_201103050800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103050800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103050800.csv"
data_siteNPP-3_camera3_201103070800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103070800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103070800.csv"
data_siteNPP-3_camera3_201103090800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103090800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103090800.csv"
data_siteNPP-3_camera3_201103110800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103110800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera3_201103110800.csv"
data_siteNPP-3_camera3_201103130800.csv	[Err

data_siteNPP-3_camera2_201208190800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208190800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208190800.csv"
data_siteNPP-3_camera2_201208200800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208200800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208200800.csv"
data_siteNPP-3_camera2_201208210800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208210800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208210800.csv"
data_siteNPP-3_camera2_201208220800.csv	[Errno 2] File /media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208220800.csv does not exist: "/media/sam/Sam's Drive/SUBSET/SQL/ALLYEARS/data_siteNPP-3_camera2_201208220800.csv"
data_siteNPP-3_camera2_201208230800.csv	[Err

Filename					Status		% Done	Frames saved


In [ ]:
print(len(others))

In [24]:
l = path_to_csvs + csvfiles[1][:-1]
df = pd.read_csv(l)
videoID = df.iloc[0][4][1:]
frame_info_fix = "frame_info_" + videoID + ".txt"
npy_name_fix = videoID + ".RESULT.npy"
npy_path = find(npy_name_fix, npy_dir)
npy = np.load(npy_path)


In [25]:
fi_path = find(frame_info_fix, frame_info_dir)
frame_info_file = create_frame_info(fi_path)

In [26]:
df
print(npy.shape)
print(len(frame_info_file))

(34,)
34


In [27]:
frame_info_ids = get_ids_from_txt(frame_info_file)
df_ids = df["detection_id"].tolist()
ids_to_remove = get_difference(df_ids, frame_info_ids)
clean_df = remove_rows(df, ids_to_remove)

In [30]:
print(npy)

[False  True  True  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False]


In [34]:
for i, item in enumerate(npy):
    if ( (item) and df.iloc[i][" specie_id"]==0):
        print(df.iloc[i][" specie_id"])

0
0
0
0
0


In [13]:
len(others)

2041

In [14]:
with open('skipped.txt', 'w') as f:
    for item in others:
        f.write("%s\n" % item)